# Elemental Substitution Pipeline
This notebook demonstrates the elemental substitution pipeline for generating new materials in Ge-Sb-Te system.

Author: ***Jianghai@BUAA***

## 1. **Data Mining from Materials Project**
Query materials from the Materials Project database that satisfy the valence constraint for Ge–Sb–Te ternary systems:
$$
2x+3y-2z=0
$$
where $x$, $y$, and $z$ are the number of Ge, Sb, and Te atoms in the formula, respectively.

**Key Output**:
A CSV file containing the filtered material entries that obey the charge balance rule based on nominal valences:

- Ge: +2
- Sb: +3
- Te: −2

In [ ]:
import os
import pandas as pd

from math import gcd
from mp_api.client import MPRester

In [ ]:
API_KEY = "type_your_API_key_here"  # Replace with your Materials Project API key

"""
            Periodic Table of Elements
"""
# Elements = ['H', 'He',
#             'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
#             'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar',
#             'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
#             'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
#             'Cs', 'Ba', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn',
#             'Fr', 'Ra', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', ' Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og',
#             'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu',
#             'Ac', 'Th', 'Pa', 'U', ' Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']


"""
            Stoichiometry satisfied the valency constraint
"""


def coprime(a, b, c):
    return gcd(a, b, c) == 1


Stoichiometry = []
for x in range(1, 20):
    for y in range(1, 20):
        for z in range(1, 20):
            if 2 * x + 3 * y - 2 * z == 0:
                if coprime(x, y, z):
                    Stoichiometry.append([x, y, z])

"""
            Data retrieval
"""
for formula in Stoichiometry:
    with MPRester(API_KEY) as mpr:
        # list_of_available_fields = mpr.materials.summary.available_fields
        # print(list_of_available_fields)
        """
        ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty',
        'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name',
        'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure',
        'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull',
        'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap',
        'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos',
        'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization',
        'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites',
        'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss',
        'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max',
        'weighted_surface_energy_EV_PER_ANG2', 'weighted_surface_energy', 'weighted_work_function',
        'surface_anisotropy', 'shape_factor', 'has_reconstructed', 'possible_species', 'has_props', 'theoretical',
        'database_IDs']
        """

        docs = mpr.materials.summary.search(formula=["*{}*{}*{}".format(formula[0], formula[1], formula[2])],
                                            fields=['material_id', 'database_IDs', 'nelements', 'chemsys',
                                                    'formula_anonymous', 'formula_pretty', 'composition',
                                                    'symmetry', 'energy_per_atom', 'formation_energy_per_atom',
                                                    'energy_above_hull', 'is_stable', 'theoretical', 'volume',
                                                    'density', 'possible_species', 'decomposes_to',
                                                    'deprecated', 'origins', 'warnings', 'last_updated',
                                                    'structure'])

        MP_Prototype_Info = [
            [doc.material_id, doc.database_IDs, doc.nelements, doc.chemsys, doc.formula_anonymous,
             doc.formula_pretty, doc.composition, doc.symmetry, doc.energy_per_atom,
             doc.formation_energy_per_atom, doc.energy_above_hull, doc.is_stable, doc.theoretical, doc.volume,
             doc.density, doc.possible_species, doc.decomposes_to, doc.deprecated, doc.origins, doc.warnings,
             doc.last_updated] for doc in docs]

    MP_Prototype_Info = pd.DataFrame(MP_Prototype_Info,
                                     columns=['material_id', 'database_IDs', 'nelements', 'chemsys',
                                              'formula_anonymous', 'formula_pretty', 'composition', 'symmetry',
                                              'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull',
                                              'is_stable', 'theoretical', 'volume', 'density', 'possible_species',
                                              'decomposes_to', 'deprecated', 'origins', 'warnings', 'last_updated'])

    if not os.path.exists(r'~\MP_Prototype_Info.csv'):
        MP_Prototype_Info.to_csv(r'~\MP_Prototype_Info.csv',
                                 mode='a', index=False)
    else:
        MP_Prototype_Info.to_csv(r'~\MP_Prototype_Info.csv',
                                 mode='a', index=False, header=False)

info = pd.read_csv(r'~\MP_Prototype_Info.csv')
info_symmetry = info['symmetry'].str.split('\'', expand=True)[[1, 3, 4, 5]]
info_spg = info_symmetry[4].str.split('=', expand=True)[1].str.split(expand=True)[0]
info_sym = pd.concat([info_symmetry, info_spg], axis=1)
info = pd.concat([info, info_sym], axis=1)
info = info.rename(columns={0: 'space_group_number', 1: 'crystal_system', 3: 'space_group', 5: 'point_group'})
info.drop([4, 'symmetry'], axis=1, inplace=True)
info.to_csv(r'~\MP_Prototype_Info.csv', mode='w', index=False)


## 2. **Element Substitution Based on Stoichiometry**
Perform element substitution in prototype structures (POSCARs) to systematically generate Ge–Sb–Te structures according to specified stoichiometries.


In [ ]:
import os
import math
import linecache

In [ ]:
def seq_element(stoichiometry, comp):
    """
            Sort the elements by stoichiometry
    """
    element_seq = []
    for idx in range(3):
        for key, value in stoichiometry.items():
            if value == comp[idx]:
                element_seq.append(key)
                stoichiometry.update({key: '0'})
                break
    return element_seq


# Access to the directory of raw files
dirs = os.listdir(r'~\Stoichiometry')
# Create directory of destination files
os.mkdir(r'~\ElementSub')
for dir in dirs:
    # Create subdirectories of destination files
    os.mkdir(r'~\ElementSub\{}'.format(dir))
    # Access to raw filenames
    files = os.listdir(r'~\Stoichiometry\\' + dir)

    for i in range(len(files)):
        # Concatenate to get filenames
        fileName = r'~\Stoichiometry\\' + dir + '\\' + files[i]

        # Bind target elements to the corresponding stoichiometry
        element = ['Ge', 'Sb', 'Te']
        composition = dir.split("-")
        stoichiometry = {element[i]: composition[i] for i in range(3)}

        # Get stoichiometry of raw files and reduce it
        comp = linecache.getline(fileName, 7).split()
        comp_int = [int(i) for i in comp]
        gcd = math.gcd(comp_int[0], comp_int[1], comp_int[2])         # Calculate the greatest common divisor (GCD)
        if gcd != 1:
            comp_int = [int(i / gcd) for i in comp_int]
            comp = [str(i) for i in comp_int]
        # Get the elements sorted correctly
        seq_ele = seq_element(stoichiometry, comp)

        # Copy the contents of raw files
        file_ini = open(fileName, "r")
        content = file_ini.read()
        file_ini.close()

        # Modify the corresponding elements and write to destination files
        with open(r'~\ElementSub\\' + dir + '\\' + '{}-{}-POSCAR'.format(files[i].split("_")[0], files[i].split("_")[1].split(".")[0]), "w") as file_processed:
            ini_ele = linecache.getline(fileName, 6).split()
            temp_ele = ['!!!', '@@@', '###']
            # Corresponding elements substitution
            for x in range(3):
                # Avoid interference
                if len(ini_ele[x]) == 2:
                    content = content.replace('{}'.format(ini_ele[x]), '{}'.format(temp_ele[x]))
            for y in range(3):
                content = content.replace('{}'.format(ini_ele[y]), '{}'.format(temp_ele[y]))
            for z in range(3):
                content = content.replace('{}'.format(temp_ele[z]), '{}'.format(seq_ele[z]))
            file_processed.write(content)
            file_processed.close()


## 3. **Permutation of Equivalent Compositions**
For stoichiometries with two or more identical atomic counts (i.e., 2–2–5), generate permutationally equivalent structures by swapping elements.

In [ ]:
dirs = os.listdir(r'~\ElementSub')

for dir in dirs:
    composition = dir.split("-")
    # Check for duplicate elements
    if len(composition) != len(set(composition)):
        # Access to the directory of permutation files
        os.mkdir(r'~\ElementSub\{}-{}-{}_Permutation'.format(*composition))
        files = os.listdir(r'~\ElementSub\\' + dir)

        for i in range(len(files)):
            fileName = r'~\ElementSub\\' + dir + '\\' + files[i]
            file_ini = open(fileName, "r")
            content = file_ini.read()
            file_ini.close()

            with open(r'~\ElementSub\{}-{}-{}_Permutation'.format(*composition) + '\\' + '{}_permutation'.format(files[i]), "w") as file_permutated:
                content = content.replace('Ge', 'G_e').replace('Sb', 'Ge').replace('G_e', 'Sb')
                file_permutated.write(content)
                file_permutated.close()


## 4. Preparing DFT Calculation Folders
Organize the substituted structures into a directory structure suitable for VASP DFT calculations.

In [ ]:
import shutil

In [ ]:
# Backup raw_folder
ini_path = r"~\ElementSub"
main_work_path = r"~\ElementSub_Calc"
shutil.copytree(ini_path, main_work_path)


dirs = os.listdir(main_work_path)

for dir in dirs:
    workdir = os.path.join(main_work_path, dir)
    files = os.listdir(workdir)

    for file in files:
        fileName = file.split("-")
        Calc_path = os.path.join(workdir, "{}-{}-{}".format(fileName[0], fileName[1], fileName[2]))
        os.mkdir(Calc_path)
        shutil.move(os.path.join(workdir, file), os.path.join(Calc_path, "POSCAR"))


## 5. **Analysis of DFT Calculation Results**
### 5.1 Convergence Check
### 5.2 Data Extraction and Processing
### 5.3 Data Selection
### 5.4 Convex Hull Analysis & Visualization

In [ ]:
import json
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt

from pymatgen.core import Structure
from pymatgen.io.vasp.outputs import Oszicar, Vasprun
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [ ]:
def check_convergence(calc_dir):
    file = os.path.join(calc_dir, 'OUTCAR')
    with open(file, 'r') as f:
        content = f.read()
        return 'reached required accuracy' in content


def data_extraction(root_dir=None, depth=2):
    if root_dir is None:
        root_dir = r'~'

    data_info = pd.DataFrame(columns=['group',
                                      'parent',
                                      'Ge', 'Sb', 'Te',
                                      'space_group_symbol',
                                      'space_group_number',
                                      'convergence',
                                      'energy_per_atom',
                                      'formation_energy_per_atom'])

    for root, dirs, files in os.walk(root_dir):
        cur_depth = root.count(os.sep) - root_dir.count(os.sep)
        if cur_depth == depth:
            if 'OUTCAR' in files and 'OSZICAR' in files and 'vasprun.xml' in files:
                group = root.split('\\')[-2]
                parent = root.split('\\')[-1]
                convergence = check_convergence(root)
                species = linecache.getline(os.path.join(root, 'POSCAR'), 6).split()
                stoichiometry = linecache.getline(os.path.join(root, 'POSCAR'), 7).split()
                composition = {'Ge': '0',
                               'Sb': '0',
                               'Te': '0'}
                composition.update({species[i]: stoichiometry[i] for i in range(len(species))})

                xml_path = os.path.join(root, 'vasprun.xml')
                oszicar_path = os.path.join(root, 'OSZICAR')
                try:
                    dataset = Vasprun(xml_path)

                    with open(os.path.join(os.path.dirname(root_dir), 'EleSub_dataset.json'), 'a',
                              newline='\n') as file:
                        json.dump(dataset.as_dict(), file)
                        file.write('\n')

                    n_atoms = len(dataset.ionic_steps[0]["structure"])
                    energy = Oszicar(oszicar_path).final_energy
                    energy_per_atom = energy / n_atoms

                    ge_energy_per_atom = -4.518072
                    sb_energy_per_atom = -4.141693
                    te_energy_per_atom = -3.141757

                    formation_energy_per_atom = (energy - (int(composition['Ge']) * ge_energy_per_atom +
                                                           int(composition['Sb']) * sb_energy_per_atom +
                                                           int(composition['Te']) * te_energy_per_atom)) / (
                                                        int(composition['Ge']) + int(composition['Sb']) + int(
                                                    composition['Te']))

                    structure = Structure.from_file(os.path.join(root, 'CONTCAR'))
                    spa = SpacegroupAnalyzer(structure)
                    spg_symbol = spa.get_space_group_symbol()
                    spg_number = spa.get_space_group_number()

                    data = {'group': group,
                            'parent': parent,
                            'Ge': int(composition['Ge']),
                            'Sb': int(composition['Sb']),
                            'Te': int(composition['Te']),
                            'space_group_symbol': spg_symbol,
                            'space_group_number': spg_number,
                            'convergence': convergence,
                            'energy_per_atom': energy_per_atom,
                            'formation_energy_per_atom': formation_energy_per_atom}

                    data = pd.DataFrame(data, index=[0])
                    data_info = pd.concat([data_info, data], ignore_index=True)
                except:
                    print('Error in {}, skipping...'.format(root))
                    continue

    data_info.to_csv(os.path.join(os.path.dirname(root_dir), 'EleSub_dataset.csv'), index=False)


def data_processing(data_path):
    data_info = pd.read_csv(data_path)
    data_info['group'] = data_info['group'].astype(str)
    data_info['convergence'] = data_info['convergence'].astype(str)
    data_info = data_info[data_info['convergence'].str.contains('True')]
    data_info.sort_values(by=['formation_energy_per_atom'], inplace=True)

    data_info[['Ge', 'Sb', 'Te']] = data_info[['Ge', 'Sb', 'Te']].astype(int)
    data_info['m'] = data_info['Ge']
    data_info['n'] = data_info['Sb'] / 2
    data_info['n'] = data_info['n'].astype(int)
    data_info['x'] = data_info['m'] / (data_info['m'] + data_info['n'])
    data_info['y'] = data_info['formation_energy_per_atom']

    data_info.to_csv(r'~\valid_data.csv', index=False)


def select_data(data_path):
    data_info = pd.read_csv(data_path)
    data_info = data_info[data_info['Ehull'] < 0.13]
    data_info = data_info[data_info['formation_energy_per_atom'] < 0]
    data_info = data_info[data_info['space_group_number'] != 1]
    data_info = data_info[~data_info['group'].str.contains('1-2-4')]
    data_info = data_info[~data_info['group'].str.contains('1-4-7')]
    data_info = data_info[~data_info['group'].str.contains('1-6-10')]
    data_info = data_info[~data_info['group'].str.contains('2-2-5')]
    data_info = data_info[~data_info['group'].str.contains('2-2-5_Permutation')]

    data_info.to_csv(r'~\EleSub_selected.csv', index=False)


def convex_hull(data_path):
    data_info = pd.read_csv(data_path)
    data = data_info[['x', 'y']]
    edge = [[0, -0.1202678],
            [1, -0.0942086]]
    data = data._append(pd.DataFrame(edge, columns=data.columns))
    data.reset_index(inplace=True, drop=True)
    pts = np.array(data)

    '''
                Convex hull determination
    '''
    hull = ConvexHull(pts)

    '''
                Set of convex hull vertices
    '''
    hull_vts_all = []
    for pt in hull.vertices:
        vertex = pts[pt]
        hull_vts_all.append(vertex)
    hull_vts_all = pd.DataFrame(hull_vts_all)

    hull_vts_negative = hull_vts_all[~(hull_vts_all[[1]] > 0).any(axis=1)]

    '''
                Equation determination
    '''
    HULL = ConvexHull(hull_vts_negative)

    HULL_Eqs = pd.DataFrame(HULL.equations)
    HULL_Eqs = np.array(HULL_Eqs)
    HULL_Eqs = np.delete(HULL_Eqs, [0], axis=0)

    '''
                Calculation of E_hull
    '''
    a, b, c, x, y = [], [], [], [], []

    for i in range(len(HULL_Eqs)):
        a_i = HULL_Eqs[i][0]
        a.append(a_i)
        b_i = HULL_Eqs[i][1]
        b.append(b_i)
        c_i = HULL_Eqs[i][2]
        c.append(c_i)

    for j in range(len(pts)):
        x_j = pts[j][0]
        x.append(x_j)
        y_j = pts[j][1]
        y.append(y_j)

    e_on_hull_set = []
    for i in range(len(pts)):
        for j in range(len(HULL_Eqs)):
            e_on_hull = -(x[i] * a[j] + c[j]) / b[j]
            e_on_hull_set.append(e_on_hull)
    e_on_hull_set = pd.DataFrame(np.array(e_on_hull_set).reshape(len(pts), len(HULL_Eqs)))
    e_on_hull = e_on_hull_set.max(axis=1)  # Select the maximum value as convex hull energy

    eform = data_info['y']
    ehull_set = []
    for i in range(len(eform)):
        ehull = eform[i] - e_on_hull[i]  # Calculation of E_hull
        ehull_set.append(ehull)

    '''
                File output
    '''
    data_info['Ehull'] = ehull_set
    data_info = pd.DataFrame(data_info)
    data_info.to_csv(r'~\EleSub_Energy_above_Hull.csv', index=False)


def plot_convex_hull(data_path):
    data = pd.read_csv(data_path)
    x1 = data['x']
    y1 = data['y']

    seeds_path = r'F:\Dataset\mp_GST_Data.csv'
    seeds = pd.read_csv(seeds_path)
    seeds = seeds[seeds['Ge'] + seeds['Sb'] / 2 * 3 == seeds['Te']]
    seeds['convergence'] = seeds['convergence'].astype(str)
    seeds = seeds[seeds['convergence'].str.contains('True')]
    seeds = seeds[['Ge', 'Sb', 'Te', 'space_group_symbol', 'space_group_number', 'formation_energy_per_atom']]
    seeds[['Ge', 'Sb', 'Te']] = seeds[['Ge', 'Sb', 'Te']].astype(int)
    seeds['m'] = seeds['Ge']
    seeds['n'] = seeds['Sb'] / 2
    seeds['n'] = seeds['n'].astype(int)
    seeds['x'] = seeds['m'] / (seeds['m'] + seeds['n'])
    seeds['y'] = seeds['formation_energy_per_atom']

    # Metastable cubic structure
    cubic_data = np.array([[0, -0.02081418],
                           [0.333333, -0.07518477],
                           [0.5, -0.052255621],
                           [0.666667, -0.071077905],
                           [1, -0.075879095]])

    x2 = seeds['x']
    y2 = seeds['y']

    hull_data = data[['x', 'y']]
    hull_seeds = seeds[['x', 'y']]
    hull_data = pd.concat([hull_data, hull_seeds])
    hull_data = hull_data[~(hull_data[['y']] > 0).any(axis=1)]

    hull_data = np.array(hull_data)
    hull = ConvexHull(hull_data)

    simplices = hull.simplices
    simplices = np.delete(simplices, [0, 1, 2, 4, 5, 6, 7, 8], axis=0)

    plt.figure(figsize=(16, 9))
    ax = plt.subplot(111)
    plt.scatter(x1, y1, color='None', marker='d', edgecolor='teal', s=100, alpha=0.25, label='Elemental Substitution')
    plt.scatter(x2, y2, color='sienna', marker='^', s=100, label='Materials Project')
    plt.scatter([0.25, 0.33333333, 0.5],
                [-0.125695, -0.1247125, -0.11877529],
                color='red', marker='d', s=100, label='On Convex Hull')
    plt.scatter(cubic_data[:, 0], cubic_data[:, 1], color='red', marker='s', s=100, label='Cubic')

    plt.rc('font', family='Times New Roman')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel("${m}/{m + n}$", fontsize=18)
    plt.ylabel("Formation Energy (eV / atom)", fontsize=18)
    plt.xlim(0, 1)
    plt.ylim(-0.2, 0.3)

    ax.set_xticks([0.142857, 0.25, 0.333333, 0.5, 0.666667, 0.75])
    ax.set_yticks([-0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2, 0.25])

    plt.legend(['Elemental Substitution', 'Materials Project', 'On Convex Hull', 'Cubic'], prop={'size': 18})

    plt.text(0.96, -0.118, 'GeTe', fontdict={'family': 'Times New Roman', 'size': 15})
    plt.text(0.22, -0.15, '$\mathregular{Ge_1Sb_6Te_{10}}$', fontdict={'family': 'Times New Roman', 'size': 15})
    plt.text(0.305, -0.15, '$\mathregular{Ge_1Sb_4Te_7}$', fontdict={'family': 'Times New Roman', 'size': 15})
    plt.text(0.305, -0.15, '$\mathregular{Ge_1Sb_4Te_7}$', fontdict={'family': 'Times New Roman', 'size': 15})
    plt.text(0.47, -0.145, '$\mathregular{Ge_1Sb_2Te_4}$', fontdict={'family': 'Times New Roman', 'size': 15})
    plt.text(0.635, -0.135, '$\mathregular{Ge_2Sb_2Te_5}$', fontdict={'family': 'Times New Roman', 'size': 15})
    plt.text(0.005, -0.138, '$\mathregular{Sb_2Te_3}$', fontdict={'family': 'Times New Roman', 'size': 15})

    for i in simplices:
        # print(hull_data[i, 0])
        # print(hull_data[i, 1])
        plt.plot(hull_data[i, 0], hull_data[i, 1], c='gold', linewidth=3)

    for i in range(len(cubic_data) - 1):
        plt.plot(cubic_data[[i, i + 1], 0], cubic_data[[i, i + 1], 1], c='gold', linewidth=3)

    plt.savefig('Elemental_Substitution_Convex_Hull.png', dpi=1200)
    plt.show()


In [ ]:
if __name__ == '__main__':
    path = r'~'
    file = os.path.join(path, r'valid_data.csv')
    data_extraction()
    data_processing(os.path.join(path, r'EleSub_dataset.csv'))
    convex_hull(os.path.join(path, r'valid_data.csv'))
    select_data(os.path.join(path, r'EleSub_Energy_above_Hull.csv'))
    plot_convex_hull(file)
